<a href="https://colab.research.google.com/github/GeoExploreTech/APP_BACKEND_NODEJS/blob/master/JSON_DATA_EXTRACTION_SCHEMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ifcopenshell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.5 MB/s eta 0:00:00


In [ ]:
import ifcopenshell
from google.colab import files
import json

In [ ]:
# Function to Upload Parse the IFC File
def load_ifc():
    """Load the IFC file and parse it for further processing."""
    # Upload IFC file
    uploaded = files.upload()
    # Save the uploaded file
    for filename in uploaded.keys():
        print(f"Uploaded file: {filename}")
    return uploaded

In [ ]:
def extract_geometry(ifc_entity):
    """
    Extracts geometry information for a given IFC entity.
    """
    geometry_data = {}
    try:
        if hasattr(ifc_entity, 'Representation') and ifc_entity.Representation:
            representations = ifc_entity.Representation.Representations
            for rep in representations:
                geometry_data[rep.RepresentationType] = [str(item) for item in rep.Items]
    except Exception as e:
        geometry_data["error"] = str(e)
    return geometry_data

def extract_ifc_hierarchy(ifc_file, root_entity):
    """
    Extracts hierarchical information from an IFC file iteratively, avoiding recursion.
    """
    stack = [root_entity]
    visited = set()
    hierarchy = {}

    while stack:
        current_entity = stack.pop()
        entity_id = current_entity.id()

        if entity_id in visited:
            continue

        visited.add(entity_id)

        # Create entity data
        entity_data = {
            "type": current_entity.is_a(),
            "id": entity_id,
            "name": getattr(current_entity, "Name", None),
            "description": getattr(current_entity, "Description", None),
            "geometry": extract_geometry(current_entity),
            "relationships": {"children": []},
        }

        hierarchy[entity_id] = entity_data

        # Process related entities
        relationships = ifc_file.get_inverse(current_entity)
        for rel in relationships:
            related_entities = (
                rel.RelatedObjects if hasattr(rel, "RelatedObjects") else
                rel.RelatedElements if hasattr(rel, "RelatedElements") else
                []
            )
            for related_entity in related_entities:
                if related_entity.id() not in visited:
                    stack.append(related_entity)
                entity_data["relationships"]["children"].append(related_entity.id())

    return hierarchy

def convert_to_cityjson(ifc_data):
    """
    Converts the extracted IFC data into CityJSON format.
    """
    cityjson = {
        "type": "CityJSON",
        "version": "1.0",
        "CityObjects": {},
        "metadata": {
            "referenceSystem": "urn:ogc:def:crs:EPSG::4326"
        },
        "vertices": [],
    }

    stack = list(ifc_data.values())
    while stack:
        current_obj = stack.pop()
        obj_id = current_obj.get("id")
        if not obj_id:
            print(f"Missing 'id' in object: {json.dumps(current_obj, indent=4)}")
            continue

        if obj_id not in cityjson["CityObjects"]:
            city_object = {
                "type": current_obj["type"],
                "attributes": {
                    "name": current_obj["name"],
                    "description": current_obj["description"]
                },
                "geometry": current_obj.get("geometry", []),
                "children": current_obj["relationships"].get("children", [])
            }

            cityjson["CityObjects"][obj_id] = city_object

    return cityjson

def parse_ifc_to_cityjson(ifc_path):
    """
    Parses an IFC file and converts it to CityJSON format.
    """
    ifc_file = ifcopenshell.open(ifc_path)
    root_elements = ifc_file.by_type("IfcProject")

    if not root_elements:
        return {"error": "No IfcProject found in the IFC file."}

    ifc_data = {}

    for root in root_elements:
        hierarchy = extract_ifc_hierarchy(ifc_file, root)
        ifc_data.update(hierarchy)

    return convert_to_cityjson(ifc_data)


In [ ]:
fileUploaded = load_ifc()


Saving bally's latest -11.ifc to bally's latest -11.ifc
Uploaded file: bally's latest -11.ifc


In [ ]:
# Input IFC file path

ifc_file_name = list(fileUploaded.keys())[0]  # Get the name of the uploaded file
output_path = "output2.json"

# Parse IFC and save as CityJSON
cityjson_data = parse_ifc_to_cityjson(ifc_file_name)

# Write CityJSON to file
with open(output_path, "w", encoding="utf-8") as cityjson_file:
    json.dump(cityjson_data, cityjson_file, indent=4)

print(f"IFC data has been saved to {output_path}")


IFC data has been saved to output2.json
